# MIMIC-IV Pharmacy Data Preprocessing
**Goal:** Clean and standardize the `pharmacy.csv` table to prepare it for Knowledge Graph construction.

### Tasks:
1. Load data and audit missing values.
2. Resolve missing medication names using `prescriptions.csv`.
3. Standardize medication strings (lowercase, stripping noise).


In [38]:
import pandas as pd
import numpy as np

# 1. Setup Display
pd.set_option('display.max_columns', None)

# 2. Define columns we actually care about for a General KG
# We are ignoring the 'specialized' ones (sliding_scale, basal_rate, etc.)
keep_cols = [
    'subject_id', 'hadm_id', 'pharmacy_id', 'poe_id', 
    'starttime', 'stoptime', 'medication', 
    'route', 'frequency', 'dispensation'
]

# 3. Load the data
df_pharm = pd.read_csv('pharmacy.csv', usecols=keep_cols, low_memory=False)

print(f"Initial Load Complete: {len(df_pharm)} rows.")

Initial Load Complete: 17847567 rows.


In [40]:
# Load the 'prescriptions' table (just the bridge columns)
df_presc = pd.read_csv('prescriptions.csv', usecols=['pharmacy_id', 'drug','ndc'], low_memory=False)

# Merge to find names for the blanks in pharmacy
df_pharm = pd.merge(df_pharm, df_presc, on='pharmacy_id', how='left')

# If 'medication' is empty, fill it with the name from 'drug'
df_pharm['medication'] = df_pharm['medication'].fillna(df_pharm['drug'])

# Now we can drop the helper column and any rows that are still missing a name
df_pharm.drop(columns=['drug'], inplace=True)
df_pharm.dropna(subset=['medication'], inplace=True)

print(f"Names Rescued. New Row Count: {len(df_pharm)}")

Names Rescued. New Row Count: 20292701


In [42]:
def clean_med_name(name):
    if pd.isna(name): return name
    name = str(name).lower().strip()
    # Remove common 'dosage form' noise that clutters KG nodes
    noise = ['(tab)', '(cap)', '(liquid)', 'iv', 'po', 'tablet', 'capsule']
    for word in noise:
        name = name.replace(word, '')
    return name.strip()

df_pharm['medication_node'] = df_pharm['medication'].apply(clean_med_name)

print(f"Unique Medication Nodes: {df_pharm['medication_node'].nunique()}")

Unique Medication Nodes: 9038


In [44]:
df_pharm['medication_node'].head()

0                     furosemide
1        ipratropium bromide neb
2                     furosemide
3               tassium chloride
4    sodium chloride 0.9%  flush
Name: medication_node, dtype: object

In [46]:
df_pharm

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,route,frequency,dispensation,ndc,medication_node
0,10000032,22595853,12775705,10000032-55,2180-05-08 08:00:00,2180-05-07 22:00:00,Furosemide,PO/NG,DAILY,Omnicell,5.107901e+10,furosemide
1,10000032,22595853,18415984,10000032-42,2180-05-07 02:00:00,2180-05-07 22:00:00,Ipratropium Bromide Neb,IH,Q6H,Omnicell,4.879801e+08,ipratropium bromide neb
2,10000032,22595853,23637373,10000032-35,2180-05-07 01:00:00,2180-05-07 09:00:00,Furosemide,PO/NG,DAILY,Omnicell,5.107901e+10,furosemide
3,10000032,22595853,26862314,10000032-41,2180-05-07 01:00:00,2180-05-07 01:00:00,Potassium Chloride,PO,ONCE,Omnicell,2.450041e+08,tassium chloride
4,10000032,22595853,30740602,10000032-27,2180-05-07 00:00:00,2180-05-07 22:00:00,Sodium Chloride 0.9% Flush,IV,Q8H,Floor Stock Item,0.000000e+00,sodium chloride 0.9% flush
...,...,...,...,...,...,...,...,...,...,...,...,...
20361148,19999987,23865745,95605092,19999987-63,2145-11-03 15:00:00,2145-11-03 18:00:00,Propofol,IV DRIP,TITRATE TO RASS,Omnicell,6.332303e+10,profol
20361149,19999987,23865745,96309533,19999987-36,2145-11-03 00:00:00,2145-11-04 13:00:00,LeVETiracetam,IV,BID,IV MED,3.380049e+08,levetiracetam
20361150,19999987,23865745,96309533,19999987-36,2145-11-03 00:00:00,2145-11-04 13:00:00,LeVETiracetam,IV,BID,IV MED,1.478904e+10,levetiracetam
20361151,19999987,23865745,97298610,19999987-192,2145-11-08 16:00:00,2145-11-11 17:00:00,Acetaminophen,PO/NG,Q6H:PRN,Omnicell,5.107900e+10,acetaminophen


In [48]:
import re

def get_base_name(name):
    if pd.isna(name): return name
    
    # 1. Lowercase and strip
    name = str(name).lower().strip()
    
    # 2. Remove anything in parentheses (e.g., "Aspirin (Bulk)" -> "Aspirin")
    name = re.sub(r'\(.*?\)', '', name)
    
    # 3. Remove common dosage/form keywords
    # This list targets the "noise" you see in the 'medication' column
    noise_patterns = [
        r'\d+mg', r'\d+ mcg', r'\d+ %', r'\d+unit',  # Strengths
        r'tab', r'cap', r'liquid', r'vial', r'syringe', # Forms
        r'iv', r'po', r'prn', r'ext', r'ec'           # Routes/Type
    ]
    for pattern in noise_patterns:
        name = re.sub(pattern, '', name)
    
    return name.strip()

# Apply the cleaning
df_pharm['medication_base'] = df_pharm['medication'].apply(get_base_name)

# Let's see the reduction
print(f"Original unique names: {df_pharm['medication'].nunique()}")
print(f"Cleaned unique names: {df_pharm['medication_base'].nunique()}")

Original unique names: 11284
Cleaned unique names: 7676


In [49]:
# Create a dictionary of the merges
merge_audit = df_pharm.groupby('medication_base')['medication'].unique()

# Convert to a DataFrame for easier viewing
df_audit = merge_audit.reset_index()
df_audit['count'] = df_audit['medication'].apply(len)

# Show the drugs that had the most "aliases" merged into them
df_audit.sort_values('count', ascending=False).head(10)

,medication_base,medication,count
0,,"[Syringe, Syringe (NS), Syringe (Chemo), (0.9%...",25
3156,heparin,"[Heparin, Heparin (IABP), Heparin (CRRT Machin...",18
1728,coq10,"[CoQ10 (300mg), CoQ10 300mg, CoQ10 200mg, coQ1...",17
653,aspirin,"[Aspirin EC, Aspirin, Aspirin (Buffered), Aspi...",16
165,acetaminophen,"[Acetaminophen, Acetaminophen IV, Acetaminophe...",12
5374,oxycodone,"[OxyCODONE (Immediate Release), OxycoDONE (Imm...",12
6581,symbicort,"[Symbicort, Symbicort (budesonide/formoterol),...",11
6643,tadalafil,"[tadalafil, tadalafiL, tadalafil (antihyperten...",10
1907,dalfampridine,"[dalfampridine, Dalfampridine (4-Aminopyridine...",9
5026,nimodipine,"[Nimodipine, NiMODipine, niMODipine, Nimodipin...",9


In [54]:
pd.set_option('display.max_columns', None)

In [56]:
df_pharm

,subject_id,hadm_id,pharmacy_id,poe_id,starttime,stoptime,medication,route,frequency,dispensation,ndc,medication_node,medication_base
0,10000032,22595853,12775705,10000032-55,2180-05-08 08:00:00,2180-05-07 22:00:00,Furosemide,PO/NG,DAILY,Omnicell,5.107901e+10,furosemide,furosemide
1,10000032,22595853,18415984,10000032-42,2180-05-07 02:00:00,2180-05-07 22:00:00,Ipratropium Bromide Neb,IH,Q6H,Omnicell,4.879801e+08,ipratropium bromide neb,ipratropium bromide neb
2,10000032,22595853,23637373,10000032-35,2180-05-07 01:00:00,2180-05-07 09:00:00,Furosemide,PO/NG,DAILY,Omnicell,5.107901e+10,furosemide,furosemide
3,10000032,22595853,26862314,10000032-41,2180-05-07 01:00:00,2180-05-07 01:00:00,Potassium Chloride,PO,ONCE,Omnicell,2.450041e+08,tassium chloride,tassium chloride
4,10000032,22595853,30740602,10000032-27,2180-05-07 00:00:00,2180-05-07 22:00:00,Sodium Chloride 0.9% Flush,IV,Q8H,Floor Stock Item,0.000000e+00,sodium chloride 0.9% flush,sodium chloride 0.9% flush
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20361148,19999987,23865745,95605092,19999987-63,2145-11-03 15:00:00,2145-11-03 18:00:00,Propofol,IV DRIP,TITRATE TO RASS,Omnicell,6.332303e+10,profol,profol
20361149,19999987,23865745,96309533,19999987-36,2145-11-03 00:00:00,2145-11-04 13:00:00,LeVETiracetam,IV,BID,IV MED,3.380049e+08,levetiracetam,levetiracetam
20361150,19999987,23865745,96309533,19999987-36,2145-11-03 00:00:00,2145-11-04 13:00:00,LeVETiracetam,IV,BID,IV MED,1.478904e+10,levetiracetam,levetiracetam
20361151,19999987,23865745,97298610,19999987-192,2145-11-08 16:00:00,2145-11-11 17:00:00,Acetaminophen,PO/NG,Q6H:PRN,Omnicell,5.107900e+10,acetaminophen,acetaminophen


In [58]:
df_presc

,pharmacy_id,drug,ndc
0,12775705,Furosemide,5.107901e+10
1,18415984,Ipratropium Bromide Neb,4.879801e+08
2,23637373,Furosemide,5.107901e+10
3,26862314,Potassium Chloride,2.450041e+08
4,30740602,Sodium Chloride 0.9% Flush,0.000000e+00
...,...,...,...
20292606,95605092,Propofol,6.332303e+10
20292607,96309533,0.9% Sodium Chloride,3.380049e+08
20292608,96309533,LeVETiracetam,1.478904e+10
20292609,97298610,Acetaminophen,5.107900e+10


In [60]:
df_presc = pd.read_csv('prescriptions.csv',)

/var/folders/y3/wh_gw8pn2_n746hflpqrrvpr0000gq/T/ipykernel_24620/2647143217.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_presc = pd.read_csv('prescriptions.csv',)


In [61]:
df_presc.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,formulary_drug_cd,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,12775705,10000032-55,55.0,P85UQ1,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,FURO40,008209,5.107901e+10,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
1,10000032,22595853,18415984,10000032-42,42.0,P23SJA,2180-05-07 02:00:00,2180-05-07 22:00:00,MAIN,Ipratropium Bromide Neb,IPRA2H,021700,4.879801e+08,2.5mL Vial,NaN,1,NEB,1,VIAL,4.0,IH
2,10000032,22595853,23637373,10000032-35,35.0,P23SJA,2180-05-07 01:00:00,2180-05-07 09:00:00,MAIN,Furosemide,FURO20,008208,5.107901e+10,20mg Tablet,NaN,20,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,26862314,10000032-41,41.0,P23SJA,2180-05-07 01:00:00,2180-05-07 01:00:00,MAIN,Potassium Chloride,MICROK10,001275,2.450041e+08,10mEq ER Tablet,NaN,40,mEq,4,TAB,1.0,PO
4,10000032,22595853,30740602,10000032-27,27.0,P23SJA,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,NACLFLUSH,NaN,0.000000e+00,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV


In [64]:
# Check what percentage of each column is missing
null_counts = df_presc.isnull().mean() * 100
print("Percentage of missing values per column:")
print(null_counts[null_counts > 0].sort_values(ascending=False))

Percentage of missing values per column:
form_rx              99.874614
doses_per_24_hrs     38.949749
gsn                  11.861598
poe_id                0.908907
poe_seq               0.908907
order_provider_id     0.327050
ndc                   0.164868
stoptime              0.154914
formulary_drug_cd     0.121054
starttime             0.107872
prod_strength         0.046312
form_unit_disp        0.046268
form_val_disp         0.046184
dose_val_rx           0.046071
dose_unit_rx          0.046066
route                 0.031731
drug                  0.000005
dtype: float64


In [65]:
df_presc['ndc']

0           5.107901e+10
1           4.879801e+08
2           5.107901e+10
3           2.450041e+08
4           0.000000e+00
                ...     
20292606    6.332303e+10
20292607    3.380049e+08
20292608    1.478904e+10
20292609    5.107900e+10
20292610    6.808403e+10
Name: ndc, Length: 20292611, dtype: float64

In [66]:
# Count unique NDC codes
unique_ndc_count = df_pharm['ndc'].nunique()

# Count unique medication base names (from our previous cleaning)
unique_med_count = df_pharm['medication_base'].nunique()

print(f"Total Unique NDC Codes: {unique_ndc_count}")
print(f"Total Unique Medication Nodes: {unique_med_count}")
print(f"Ratio: {unique_ndc_count / unique_med_count:.2f} NDCs per medication name.")

Total Unique NDC Codes: 6588
Total Unique Medication Nodes: 7676
Ratio: 0.86 NDCs per medication name.


In [70]:
# 1. Filter for rows where NDC is NaN but medication_base exists
missing_ndc_df = df_pharm[df_pharm['ndc'].isna() & df_pharm['medication_base'].notna()]

# 2. Group by medication_base to see which drugs are most frequently missing NDCs
missing_ndc_counts = missing_ndc_df['medication_base'].value_counts().reset_index()
missing_ndc_counts.columns = ['medication_base', 'missing_ndc_count']

# 3. Calculate what percentage of that drug's total entries are missing NDCs
total_counts = df_pharm['medication_base'].value_counts().reset_index()
total_counts.columns = ['medication_base', 'total_count']

# Merge to see the percentage
missing_report = pd.merge(missing_ndc_counts, total_counts, on='medication_base')
missing_report['percent_missing_ndc'] = (missing_report['missing_ndc_count'] / missing_report['total_count']) * 100

print("Top 10 Medications missing NDC most often:")
print(missing_report.head(10))

Top 10 Medications missing NDC most often:
        medication_base  missing_ndc_count  total_count  percent_missing_ndc
0          insulin pump               2882         2906            99.174123
1  sodium chloride 0.9%               1723       232184             0.742084
2                profol                528        68022             0.776219
3             symbicort                516          698            73.925501
4         phenylephrine                335        91101             0.367724
5        norepinephrine                276        67858             0.406732
6            venetoclax                249         1563            15.930902
7             ibrutinib                240          297            80.808081
8      tassium chloride                238       695066             0.034241
9                rytary                220          220           100.000000


In [74]:
# Filter for rows with no NDC, then count the medication names
missing_ndc_list = df_pharm[df_pharm['ndc'].isna()]['medication_base'].value_counts()

print("Top 50 Medications missing NDC codes:")
print(missing_ndc_list.head(50))

Top 50 Medications missing NDC codes:
medication_base
insulin pump               2882
sodium chloride 0.9%       1723
profol                      528
symbicort                   516
phenylephrine               335
norepinephrine              276
venetoclax                  249
ibrutinib                   240
tassium chloride            238
rytary                      220
fentanyl                    218
ruxolitinib                 206
profol /100ml 100ml         175
combigan                    164
midazolam                   161
melatonin                   161
levemir                     157
lumigan                     143
acetaminophen               141
lorazepam                   141
anoro ellipta               135
natinib                     133
lamictal xr                 119
fentanyl citrate            114
pulmicort flexhaler         113
alfuzosin                   105
revlimid                    104
linzess                     104
ondansetron                 104
lenalidomide      